In [7]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

# load the options data
options_data = pd.read_csv('C:\\Users\\mason\\IV\\SOXL\\soxl-options-chain-data.csv')

# Load the OHLCV data
ohlcv_data = pd.read_csv('C:\\Users\\mason\\IV\\SOXL\\soxl-ohlcv-data.csv')

In [12]:
# Check the shape of options_data before removing duplicates
print(f"Shape of options_data before removing duplicates: {options_data.shape}")

# Drop duplicate rows in options_data
options_data = options_data.drop_duplicates()

# Check the shape of options_data after removing duplicates
print(f"Shape of options_data after removing duplicates: {options_data.shape}")


Shape of options_data before removing duplicates: (4100, 34)
Shape of options_data after removing duplicates: (410, 34)


In [8]:
# Rename the 'date' column in the options chain data to 'Date'
options_data.rename(columns={'date': 'Date'}, inplace=True)

In [15]:
# Sort options_data by date column in descending order
options_data = options_data.sort_values(by='Date', ascending=False)

# Display the first few rows to verify the sorting
print("First few rows of options_data after sorting by date (descending):")
print(options_data.head())


First few rows of options_data after sorting by date (descending):
      type  strike  open  high   low  last  last_size  change  pctchange  \
0      put    13.5  2.80  4.64  2.73  4.21          1     0.0        0.0   
2051  call    13.0  0.49  0.60  0.15  0.28          5     0.0        0.0   
1      put    13.0  2.47  4.70  2.16  3.86          2     0.0        0.0   
2060  call     8.5  2.76  2.76  1.28  1.86          3     0.0        0.0   
2059  call     9.0  2.52  2.55  1.00  1.54          1     0.0        0.0   

      previous  ... volatility_pctchange  theoretical     delta     gamma  \
0         4.21  ...                  0.0         4.21 -0.786541  0.086863   
2051      0.28  ...                  0.0         0.28  0.198770  0.096414   
1         3.86  ...                  0.0         3.86 -0.730139  0.091799   
2060      1.86  ...                  0.0         1.86  0.706098  0.106475   
2059      1.54  ...                  0.0         1.54  0.645635  0.119616   

         thet

In [16]:
options_data.to_csv("soxl-options-data-clean.csv", index=False)

In [18]:
# Convert the 'Date' column in the options chain data to datetime format
options_data['Date'] = pd.to_datetime(options_data['Date'])
ohlcv_data['Date'] = pd.to_datetime(ohlcv_data['Date'])

# Attempt the merge again
merged_data = pd.merge(ohlcv_data, options_data, on='Date', how='inner')

# Display the first few rows of the merged data
merged_data.head()

,Date,Close/Last,Volume,Open,High,Low,type,strike,open,high,...,volatility_pctchange,theoretical,delta,gamma,theta,vega,rho,vol_oi_ratio,dte,midpoint
0,2025-04-10,9.63,686331800,10.915,11.14,8.35,put,13.5,2.80,4.64,...,0.0,4.21,-0.786541,0.086863,-0.068674,0.003879,-0.001734,0.0,7,4.13
1,2025-04-10,9.63,686331800,10.915,11.14,8.35,call,13.0,0.49,0.60,...,0.0,0.28,0.198770,0.096414,-0.057829,0.003720,0.000313,0.0,7,0.26
2,2025-04-10,9.63,686331800,10.915,11.14,8.35,put,13.0,2.47,4.70,...,0.0,3.86,-0.730139,0.091799,-0.084106,0.004408,-0.001665,0.0,7,3.78
3,2025-04-10,9.63,686331800,10.915,11.14,8.35,call,8.5,2.76,2.76,...,0.0,1.86,0.706098,0.106475,-0.080186,0.004593,0.000947,0.0,7,1.87
4,2025-04-10,9.63,686331800,10.915,11.14,8.35,call,9.0,2.52,2.55,...,0.0,1.54,0.645635,0.119616,-0.083226,0.004962,0.000897,0.0,7,1.55


In [19]:
merged_data.to_csv("merged_data.csv", index=False)

In [20]:
merged_data.columns

Index(['Date', 'Close/Last', 'Volume', 'Open', 'High', 'Low', 'type', 'strike',
       'open', 'high', 'low', 'last', 'last_size', 'change', 'pctchange',
       'previous', 'bid', 'bid_size', 'ask', 'ask_size', 'moneyness', 'volume',
       'volume_change', 'volume_pctchange', 'open_interest',
       'open_interest_change', 'open_interest_pctchange', 'volatility',
       'volatility_change', 'volatility_pctchange', 'theoretical', 'delta',
       'gamma', 'theta', 'vega', 'rho', 'vol_oi_ratio', 'dte', 'midpoint'],
      dtype='object')

In [23]:
import pandas as pd
import numpy as np

# Read the merged data and ensure proper date parsing
df = pd.read_csv('C:\\Users\\mason\\IV\\SOXL\\merged_data.csv')
df['Date'] = pd.to_datetime(df['Date'])

# Convert relevant columns to numeric (if they aren't already)
cols_to_numeric = ['Close/Last', 'Open', 'High', 'Low', 'Volume', 'volume']
for col in cols_to_numeric:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Since the OHLCV values are the same for every option on a given date,
# drop duplicates to calculate technical indicators on a daily basis.
df_ohlc = df[['Date', 'Close/Last', 'Open', 'High', 'Low', 'Volume']].drop_duplicates('Date').sort_values('Date').reset_index(drop=True)

# --- Technical Indicators Calculation ---

# 1. Daily Returns based on close price
df_ohlc['daily_return'] = df_ohlc['Close/Last'].pct_change()

# 2. Simple Moving Averages (SMA)
df_ohlc['sma_5'] = df_ohlc['Close/Last'].rolling(window=5, min_periods=5).mean()
df_ohlc['sma_10'] = df_ohlc['Close/Last'].rolling(window=10, min_periods=10).mean()

# 3. Exponential Moving Averages (EMA) for MACD calculation
df_ohlc['ema_12'] = df_ohlc['Close/Last'].ewm(span=12, adjust=False).mean()
df_ohlc['ema_26'] = df_ohlc['Close/Last'].ewm(span=26, adjust=False).mean()

# 4. MACD and Signal line
df_ohlc['macd'] = df_ohlc['ema_12'] - df_ohlc['ema_26']
df_ohlc['macd_signal'] = df_ohlc['macd'].ewm(span=9, adjust=False).mean()

# 5. RSI (14-day)
def compute_rsi(series, period=14):
    delta = series.diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=period, min_periods=period).mean()
    avg_loss = loss.rolling(window=period, min_periods=period).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

df_ohlc['rsi_14'] = compute_rsi(df_ohlc['Close/Last'], period=14)

# 6. ATR (14-day) calculation: Average True Range using High, Low, and previous Close
df_ohlc['prev_close'] = df_ohlc['Close/Last'].shift(1)
df_ohlc['tr'] = df_ohlc[['High', 'Low', 'prev_close']].apply(
    lambda row: max(
        row['High'] - row['Low'], 
        abs(row['High'] - row['prev_close']), 
        abs(row['Low'] - row['prev_close'])
    ), axis=1)
df_ohlc['atr_14'] = df_ohlc['tr'].rolling(window=14, min_periods=14).mean()

# 7. Intraday Range Percentage: (High - Low) / Open
df_ohlc['intraday_range_pct'] = (df_ohlc['High'] - df_ohlc['Low']) / df_ohlc['Open']

# Select the features to merge back
tech_features = [
    'Date', 'daily_return', 'sma_5', 'sma_10', 'ema_12', 'ema_26',
    'macd', 'macd_signal', 'rsi_14', 'atr_14', 'intraday_range_pct'
]
df_features = df_ohlc[tech_features]

# --- Merge the engineered features back into the merged dataset ---

# Because the original merged data has multiple rows per Date (one for each options data point),
# a left-merge will add the same computed technical indicator values to all rows with the same Date.
df_engineered = pd.merge(df, df_features, on='Date', how='left')


# Preview the resulting dataframe
print(df_engineered.head())


        Date  Close/Last     Volume    Open   High   Low  type  strike  open  \
0 2025-04-10        9.63  686331800  10.915  11.14  8.35   put    13.5  2.80   
1 2025-04-10        9.63  686331800  10.915  11.14  8.35  call    13.0  0.49   
2 2025-04-10        9.63  686331800  10.915  11.14  8.35   put    13.0  2.47   
3 2025-04-10        9.63  686331800  10.915  11.14  8.35  call     8.5  2.76   
4 2025-04-10        9.63  686331800  10.915  11.14  8.35  call     9.0  2.52   

   high  ...  daily_return  sma_5  sma_10     ema_12    ema_26      macd  \
0  4.64  ...     -0.245889  9.706  12.413  12.896443  15.56184 -2.665398   
1  0.60  ...     -0.245889  9.706  12.413  12.896443  15.56184 -2.665398   
2  4.70  ...     -0.245889  9.706  12.413  12.896443  15.56184 -2.665398   
3  2.76  ...     -0.245889  9.706  12.413  12.896443  15.56184 -2.665398   
4  2.55  ...     -0.245889  9.706  12.413  12.896443  15.56184 -2.665398   

   macd_signal     rsi_14    atr_14  intraday_range_pct  
0   

In [24]:
# Rename volume columns to be more descriptive
# 'Volume' (uppercase) is the stock volume from OHLC data
# 'volume' (lowercase) is the options volume
df_engineered = df_engineered.rename(columns={
    'Volume': 'ohlcv_volume',
    'volume': 'option_volume'
})

# Update the volume ratio calculation to use the new column names
df_engineered['options_to_ohlcv_volume_ratio'] = df_engineered['option_volume'] / df_engineered['ohlcv_volume']

# Display the updated column names
print(df_engineered.columns)


Index(['Date', 'Close/Last', 'ohlcv_volume', 'Open', 'High', 'Low', 'type',
       'strike', 'open', 'high', 'low', 'last', 'last_size', 'change',
       'pctchange', 'previous', 'bid', 'bid_size', 'ask', 'ask_size',
       'moneyness', 'option_volume', 'volume_change', 'volume_pctchange',
       'open_interest', 'open_interest_change', 'open_interest_pctchange',
       'volatility', 'volatility_change', 'volatility_pctchange',
       'theoretical', 'delta', 'gamma', 'theta', 'vega', 'rho', 'vol_oi_ratio',
       'dte', 'midpoint', 'daily_return', 'sma_5', 'sma_10', 'ema_12',
       'ema_26', 'macd', 'macd_signal', 'rsi_14', 'atr_14',
       'intraday_range_pct', 'options_to_ohlcv_volume_ratio'],
      dtype='object')


In [25]:
# Optionally, save the engineered dataset to a new CSV file:
df_engineered.to_csv('merged_data_engineered.csv', index=False)